# 음성 분류 경진대회 두 번째 베이스라인

이번 대회는 음성 데이터를 이용하여 0~9까지 숫자를 분류하는 대회입니다.

딥러닝을 이용하여 기초적인 CNN 모델로 숫자를 분류해보겠습니다.

# 분석 환경 준비

* 오디오 전처리를 위한 라이브러리

In [1]:
import librosa 
import librosa.display as dsp
from IPython.display import Audio

* 데이터 전처리를 위한 라이브러리

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

* GPU 할당

In [4]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당

* 모델의 재현성을 위하여 random seed 고정

In [5]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(929)

# 데이터 불러오기
분석하려는 데이터를 가져오는 작업

먼저 csv 파일을 불러와서 label이 어떻게 되어있는지 살펴보겠습니다.

In [6]:
import pandas as pd
train = pd.read_csv('data/train.csv')
train.head()

,id,file_name,label
0,0,553.wav,0
1,1,302.wav,3
2,2,056.wav,8
3,3,108.wav,6
4,4,262.wav,1


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         400 non-null    int64 
 1   file_name  400 non-null    object
 2   label      400 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ KB


label이 int 타입으로 되어있는 것을 확인할 수 있습니다.

# 데이터 전처리

## 음성 데이터 Load

그럼 이제 음성데이터를 한번 살펴봅시다.

소리는 기본적으로 특정 주파수를 가지는 sin함수들의 합입니다.

특정 시간에 주파수 성분이 어떻게 구성되어 있는지 확인할 수 있는데요. 

음성 데이터 분석을 할 때 주파수 분석 기법을 많이 사용합니다. (파형 자체를 이용하기도 합니다!) 

주파수 분석은 크게 3단계로 이루어지는데, 이번 포스팅에서는 그 중에서도


음성 데이터를 학습하기 위해서는 아날로그 데이터로 되어있는 음성 데이터를 디지털 신호로 변환해야 합니다. 

따라서 librosa 라이브러리를 사용해 음성 데이터를 load 를 해봅시다.

우선 예시로 train 음성 데이터의 첫번째 데이터를 load 해보겠습니다.

In [8]:
data, sample_rate = librosa.load('data/train/001.wav', sr = 16000)
print('sample_rate:', sample_rate, ', audio shape:', data.shape)
print('length:', data.shape[0]/float(sample_rate), 'secs')

sample_rate: 16000 , audio shape: (10192,)
length: 0.637 secs


sampling rate의 의미는 초당 16000개(16000Hz 주파수)의 샘플을 가지고 있는 데이터라는 의미입니다. (1초에 음성 신호를 16000번 sampling) 

sampling rate의 defult값은 22050Hz인데, 16000Hz으로 설정한 이유는 사람의 목소리는 대부분 16000Hz 안에 포함된다고 합니다.

또한 audio shape와 sampling rate를 이용해서 오디오 길이 계산을 할 수 있습니다.

계산 결과, 길이는 0.637 secs 가 되는군요!

그럼 디지털 신호와 라벨을 포함한 데이터프레임을 생성하겠습니다.

In [9]:
def train_dataset():
    folder = "data/train/"
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            class_label = int(train[train.file_name == file].label)
            dataset.append([data,class_label])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data','label'])

In [10]:
def test_dataset():
    folder = "data/test/"
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            
            dataset.append([data, file])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data', 'file_name'])

In [11]:
train_wav = train_dataset()
test_wav = test_dataset()

100%|██████████| 401/401 [00:11<00:00, 33.42it/s]


Dataset 생성 완료


100%|██████████| 200/200 [00:05<00:00, 34.54it/s]

Dataset 생성 완료


In [12]:
train_wav.head()

,data,label
0,"[0.00029359042, 0.0004896918, 0.0004225315, 0....",0
1,"[0.00021014328, 0.0003359131, 0.00028251947, 0...",1
2,"[-5.409059e-05, -0.00013994443, -0.00013878023...",2
3,"[-1.7762668e-05, -5.3217824e-05, -1.977646e-05...",4
4,"[-9.07404e-05, -0.00016544455, -0.00015305405,...",6


이번 대회에서 음성은 각각 다른 길이를 갖고 있습니다.      

baseline 코드에서는 음성 중 길이가 가장 작은 길이의 데이터를 기준으로 데이터를 잘라서 사용하겠습니다.

In [13]:
train_x = np.array(train_wav.data)
test_x = np.array(test_wav.data)

In [14]:
# 음성의 길이 중 가장 작은 길이를 구합니다.

def get_mini(data):

    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)

    return mini

train_mini = get_mini(train_x)
test_mini = get_mini(test_x)

#음성들의 길이를 맞춰줍니다.

mini = np.min([train_mini, test_mini])

가장 작은 길이는 다음과 같습니다.

In [15]:
print('가장 작은 길이 :', mini)

가장 작은 길이 : 5711


data의 길이를 가장 작은 길이에 맞춰 잘라줍니다.

In [16]:
def set_length(data, d_mini):

    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)

    return result

train_x = set_length(train_x, mini)
test_x = set_length(test_x, mini)

이렇게 전처리 된 음성 데이터 특징 모양은 다음과 같습니다.

In [17]:
print('train :', train_x.shape)
print('test :', test_x.shape)

train : (400, 5711)
test : (200, 5711)


## 음성 데이터 특징 추출

음성 데이터를 load 했으면 이 음성 데이터의 특징을 추출해야 합니다.

음성 raw data를 그대로 사용하면 파라미터가 너무 많아지기도 하고 데이터 용량이 너무 커집니다.

따라서 입력된 신호에서 노이즈 및 배경 소리로 부터 실제로 유용한 소리의 특징을 추출하는 것이죠.


### 퓨리에 변환

음성 데이터를 분석하기 위하여 주파수(frequency)를 성분을 뽑아내야 한다면 퓨리에 변환(Fourier Transform)을 해야 합니다.

퓨리에 변환은 간단히 설명하자면 **'입력 신호를 다양한 주파수를 가지는 주기함수들로 분해하는 것'** 입니다.

주기함수들을 분해함으로써 음성 데이터에서 노이즈 및 배경 소리로 부터 실제로 유용한 소리의 데이터, 즉 특징을 추출하는 것 입니다.


### MFCC (Mel-frequency cepstral coefficients)


하지만 음성데이터 전체를 퓨리에 변환을 한다면,

예를들어 "안녕하세요"라고 하더라도, 어떤 사람은 1초, 어떤 사람은 3초가 걸릴 수도 있습니다.

따라서 이 천차만별인 길이에 대하여 같은 "안녕하세요"라는 음성이라고 학습시키기는 어려울 것입니다.

위와 같은 문제를 해결하기 위하여 MFCC (Mel-frequency cepstral coefficients) 알고리즘을 이용합니다.

MFCC는 음성데이터를 특징벡터화 해주는 알고리즘입니다.

입력된 소리 전체를 대상으로 하는 것이 아니라, 

사람이 인지하기 좋은 **Mel-scale로 음성데이터를 모두 20~40ms로 나누어** 이 구간에 대한 스펙트럼을 분석하여 

**퓨리에 변환**을 한 특징 추출 방법입니다.

사람의 음성은 20~40ms 사이에서는 음소(현재 내고 있는 발음)가 바뀔 수 없다는 연구결과들을 기반으로 

음소는 해당 시간내에 바뀔 수 없다고 가정합니다.

따라서 MFCC에서는 음성데이터를 모두 20~40ms 단위로 쪼개고, 

쪼갠 단위에 대해서 Mel 값을 뽑아서 Feature로 사용합니다.

파이썬에서는 librosa.feature.mfcc(wav) 메소드 이용하여 

손쉽게 MFCC 알고리즘을 이용합니다.


#### **Mel-scale**

**Mel은 사람의 달팽이관을 모티브로 따온 값**입니다.

달팽이관은 주파수가 낮은 대역에서는 변화하는 주파수를 잘 감지하는데, 

주파수가 높은 대역에서는 주파수 감지를 잘 하지 못합니다. (주파수 간격이 넓어짐)

이러한 원리를 이용해서 filter, scaling 해줄 수 있는데, 이때 이 기준을 Mel-Scale 이라고 합니다.


#### Argument 

* y : audio data
   
*  sr : sampling rate
   
* n_mfcc :  return 될 mfcc의 개수를 정해주는 파라미터. 더 다양한 데이터 특징을 추출하기 위해서 증가 시킵니다.
   
* n_fft : frame의 length를 결정하는 파라미터. n_fft를 설정하면 window size가 디폴트 값으로 n_fft가 됩니다.        

    사람의 목소리는 대부분 16000Hz 안에 포함이 되는데, 일반적으로 자연어 처리에서는 음성을 25m의 크기를 기본으로 하고 있습니다.        
    
    (ex. 16000Hz인 음성에서는 25m의 음성의 크기를 가지고 있으면 n_fft는 16000 * 0.025 = 400 (sampling rate * frame_length = n_fft)가 됩니다.)
        
* hop_length : 윈도우 길이를 나타냅니다. 길이만큼 옆으로 가면서 데이터를 읽습니다.

    hop_length도 마찬가지로 window 간의 거리이므로 sampling rate * frame_stride 가 됩니다.

그럼 예시로, train_x에서 첫번째 음성의 MFCC 특징을 추출해보겠습니다.

In [18]:
extracted_features = librosa.feature.mfcc(y=train_x[0], sr=16000, n_mfcc=40)
extracted_features.shape

(40, 12)

출력값은 (n_mfcc, time_step) 으로 구성되어 있습니다.

그럼 train 셋 데이터 전체를 MFCC 알고리즘을 이용하여 특징추출을 해보겠습니다.

In [19]:
def preprocess_dataset(data):
    mfccs = []
    for i in data:
        extracted_features = librosa.feature.mfcc(y=i,
                                              sr=16000,
                                              n_mfcc=40)
        mfccs.append(extracted_features)
            
    return mfccs

In [20]:
train_mfccs = preprocess_dataset(train_x)
train_mfccs = np.array(train_mfccs)
train_mfccs = train_mfccs.reshape(-1, train_mfccs.shape[1], train_mfccs.shape[2], 1)
#test_x = test_x.reshape(-1, test_x.shape[1], test_x.shape[2], 1)

In [21]:
np.array(train_mfccs).shape

(400, 40, 12, 1)


### CustomDataset

전체 dataset을 구성하기위하여 CustomDataset 클래스를 생성합니다.

In [22]:
import torchvision.datasets as datasets # 데이터셋 집합체
import torchvision.transforms as transforms # 변환 툴

from torch.utils.data import DataLoader # 학습 및 배치로 모델에 넣어주기 위한 툴
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, X, y, train_mode=True, transforms=None): #필요한 변수들을 선언
        self.X = X
        self.y = y
        self.train_mode = train_mode
        self.transforms = transforms

    def __getitem__(self, index): #index번째 data를 return
        X = self.X[index]
        
        if self.transforms is not None:
            X = self.transforms(X)

        if self.train_mode:
            y = self.y[index]
            return X, y
        else:
            return X
    
    def __len__(self): #길이 return
        return len(self.X)

#### Train / Validation Split

그럼 학습시킬 데이터 셋과 검증할 데이터 셋을 분리해주도록 하겠습니다.

3:1 비율로 train과 vaildation set을 나누겠습니다

In [23]:
train_X = train_mfccs[:300]
vali_X = train_mfccs[300:]

In [24]:
train_y = train_wav.label[:300]
vali_y = train_wav.label[300:].reset_index(drop = True)

## Dataloader
Dataloader class는 batch기반의 딥러닝모델 학습을 위해서 mini batch를 만들어주는 역할을 합니다. 

dataloader를 통해 dataset의 전체 데이터가 batch size로 나뉘게 됩니다. 

만들었던 dataset을 input으로 넣어주면 여러 옵션(데이터 묶기, 섞기, 알아서 병렬처리)을 통해 batch를 만들어 내는 것입니다.

In [25]:
# 에포크 설정
num_epochs = 100

# 배치 사이즈 설정
batch_size = 10

#만든 train dataset를 DataLoader에 넣어 batch 만들기
train_dataset = CustomDataset(X=train_X, y=train_y)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

vali_dataset = CustomDataset(X=vali_X, y=vali_y)
vali_loader = DataLoader(vali_dataset, batch_size = batch_size, shuffle=False)

In [26]:
train_batches = len(train_loader)
vali_batches = len(vali_loader)

print('/ total train batches :', train_batches)
print('/ total valid batches :', vali_batches)

/ total train batches : 30
/ total valid batches : 10


## 모델 구조 정의

이제 CNN 모델을 학습시키기 위한 데이터 셋이 준비되었다면   
모델 구조를 설정하는 단계로 넘어가겠습니다.

In [27]:
from tqdm.auto import tqdm
import torch.nn as nn # 신경망들이 포함됨

class CNNclassification(torch.nn.Module):
    def __init__(self):
        super(CNNclassification, self).__init__()
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(40, 10, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(10, 100, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(200, 300, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.fc_layer = nn.Sequential( 
            nn.Linear(300, 10) #fully connected layer(ouput layer)
        )    
        
    def forward(self, x):
        
        x = self.layer1(x) #1층
        
        x = self.layer2(x) #2층
         
        x = self.layer3(x) #3층
        
        x = self.layer4(x) #4층
        
        x = torch.flatten(x, start_dim=1) # N차원 배열 -> 1차원 배열
        
        out = self.fc_layer(x)
        return out

## 모델 학습

이제 모델 학습을 하기 위해 매개변수를 정의해보도록 하겠습니다.

In [28]:
import torch.optim as optim # 최적화 알고리즘들이 포함힘

model = CNNclassification().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(params = model.parameters(), lr = 1e-3 )
scheduler = None

In [29]:
model(torch.rand(10, 40, 12, 1).to(device))

tensor([[ 6.3272e-02,  2.6929e-02, -9.6571e-03, -5.1214e-02,  1.6838e-02,
         -5.0106e-02, -5.2381e-02,  1.3524e-03,  4.0278e-02,  4.3555e-02],
        [ 6.1824e-02,  2.7923e-02, -1.1238e-02, -5.1971e-02,  1.5232e-02,
         -5.0787e-02, -5.2307e-02,  2.1474e-03,  3.7560e-02,  4.4356e-02],
        [ 6.1668e-02,  2.5830e-02, -1.0102e-02, -4.9540e-02,  1.5662e-02,
         -5.3129e-02, -5.3434e-02,  2.5749e-03,  3.8416e-02,  4.4608e-02],
        [ 6.3229e-02,  2.9607e-02, -1.0866e-02, -5.2343e-02,  1.5266e-02,
         -5.0899e-02, -5.2491e-02,  3.6425e-03,  3.8196e-02,  4.4868e-02],
        [ 6.3534e-02,  2.5505e-02, -1.0897e-02, -5.0993e-02,  1.7650e-02,
         -5.3773e-02, -5.1939e-02, -2.8022e-04,  3.5802e-02,  4.3764e-02],
        [ 6.3745e-02,  2.7125e-02, -1.0257e-02, -5.2642e-02,  1.4189e-02,
         -5.3396e-02, -5.3332e-02,  3.5185e-05,  3.8465e-02,  4.5951e-02],
        [ 6.3431e-02,  2.6569e-02, -1.1134e-02, -5.2119e-02,  1.5154e-02,
         -5.3788e-02, -5.2260e-0

### Loss Function

손실함수로는 classification 문제이기 때문에 'CrossEntropyLoss'를 사용했습니다.

손실함수는 실제 값과 모델이 예측한 값의 거리를 출력하는 함수 입니다.

쉽게 말해 손실함수는 모델의 예측이 얼마나 틀렸는지를 알려주는 함수 입니다.

이 때 "모델의 예측이 얼마나 틀렸는지" 를 어떻게 정의하느냐에 따라 어떤 Loss Function 을 사용할 지가 정해지는 것 입니다.

### Optimizer 

최적화 함수로는 확률적 경사 하강법인 'SGD(Stochastic Gradient Descent)'를 사용했습니다. 

Optimizer는 학습 데이터(Train data)셋을 이용하여 모델을 학습 할 때 데이터의 실제 결과와 모델이 예측한 결과를 기반으로 잘 줄일 수 있게 만들어주는 역할을 합니다.

여기서 learning rate, 학습률은 얼마나 빠른속도로 이동할것이냐 입니다.

learning rate를 엄청 크게 설정한다면 원하는 값까지 빠르게 도달할 수 있지만 자칫하면 오히려 최소값을 계산하도록 수렴하지 못합니다.

반면 너무 작은 경우는 시간이 매우 오래걸립니다.

따라서 적절한 learning rate설정이 중요합니다.

이제 train 메소드를 통하여 train을 학습 시켜 vaildation으로 평가하는 메소드를 작성해보겠습니다.

In [30]:
from tqdm.auto import tqdm

def train(model, optimizer, train_loader, scheduler, device): 
    model.to(device)
    n = len(train_loader)
    best_acc = 0
    
    for epoch in range(1,num_epochs): #에포크 설정
        model.train() #모델 학습
        running_loss = 0.0
        
        for wav, label in tqdm(iter(train_loader)):
            
            wav, label = wav.to(device), label.to(device) #배치 데이터
            optimizer.zero_grad() #배치마다 optimizer 초기화
        
            # Data -> Model -> Output
            logit = model(wav) #예측값 산출
            loss = criterion(logit, label) #손실함수 계산
            
            # 역전파
            loss.backward() #손실함수 기준 역전파 
            optimizer.step() #가중치 최적화
            running_loss += loss.item()
             
        print('[%d] Train loss: %.10f' %(epoch, running_loss / len(train_loader)))
        
        if scheduler is not None:
            scheduler.step()
            
            
        #Validation set 평가
        model.eval() #evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
        vali_loss = 0.0
        correct = 0
       
        with torch.no_grad(): #파라미터 업데이트 안하기 때문에 no_grad 사용
            for wav, label in tqdm(iter(vali_loader)):
                
                wav, label = wav.to(device), label.to(device)
                logit = model(wav)
                vali_loss += criterion(logit, label)
                pred = logit.argmax(dim=1, keepdim=True)  #10개의 class중 가장 값이 높은 것을 예측 label로 추출
                correct += pred.eq(label.view_as(pred)).sum().item() #예측값과 실제값이 맞으면 1 아니면 0으로 합산
        vali_acc = 100 * correct / len(vali_loader.dataset)
        print('Vail set: Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(vali_loss / len(vali_loader), correct, len(vali_loader.dataset), 100 * correct / len(vali_loader.dataset)))
        
        #베스트 모델 저장
        if best_acc < vali_acc:
            best_acc = vali_acc
            torch.save(model.state_dict(), 'data/saved/best_model2.pth') #이 디렉토리에 best_model.pth을 저장
            print('Model Saved.')

classification 문제이기 때문에 평가지표로는 Accuarcy를 사용하여 모델의 정확도를 산출하였습니다.

### epoch
딥러닝에서 epoch는 전체 트레이닝 셋이 신경망을 통과한 횟수입니다.

1-epoch는 전체 트레이닝 셋이 하나의 신경망에 적용되어 순전파와 역전파를 통해 신경망을 한 번 통과했다는 뜻입니다.

epoch 은 많을 수록 학습이 잘되는 것이 아닙니다.

epoch 이 너무 적을 경우 학습이 덜 이루어지는 경우가 있고, epoch 이 너무 많을 경우 과적합이 되는 경우가 있습니다.

따라서 적절한 epoch 을 설정해 주어야 합니다.

이때 validation loss 와 accuracy 은 epoch 을 언제 중단 할지 모니터링 하는 용도로 사용되기도 합니다.

### batch size
batch size란 cpu 또는 gpu 연산 시, 하드웨어로 로드되는 데이터의 개수 입니다.

본인의 컴퓨팅 환경에 따라 batch size 를 조절하는 것이 좋습니다.

gpu 를 사용하는 경우 본인의 gpu 메모리 용량을 고려하여 batch size 를 설정해 주어야 합니다.

batch size 는 모델 학습 과정에 영향을 끼치기도 합니다.

따라서 하드웨어 상황을 고려하면서도 학습 과정 또한 고려하여 batch size 를 설정해 주어야 합니다.

### Backpropagation (역전파)
Backpropagation 오차 역전파법이라고도 하며 예측값과 실제값의 차이인 오차를 계산하고,

이것을 다시 역으로 전파하여 가중치를 수정하여 오차가 작아지는 방향으로 일정 횟수를 반복해 수정하는 방법입니다.

이때, 역전파 과정에서는 앞서 언급했던 최적화 함수를 이용합니다.

In [31]:
train(model, optimizer, train_loader, scheduler, device)

  0%|          | 0/30 [00:00<?, ?it/s]

[1] Train loss: 2.5879614949


  0%|          | 0/10 [00:00<?, ?it/s]

Vail set: Loss: 2.2402, Accuracy: 18/100 ( 18%)

Model Saved.


  0%|          | 0/30 [00:00<?, ?it/s]

...

Vail set: Loss: 0.6749, Accuracy: 82/100 ( 82%)



  0%|          | 0/30 [00:00<?, ?it/s]

[99] Train loss: 0.0950146261


  0%|          | 0/10 [00:00<?, ?it/s]

Vail set: Loss: 0.7459, Accuracy: 81/100 ( 81%)



에포크가 83일때 Vaildation Accuracy가 83%로 best_model에 선정되어 저장되었습니다.

## 추론하기

이제 학습된 best_model을 가지고 test 셋의 라벨을 추론해보도록 하겠습니다.

우선 test셋도 똑같이 특징을 추출한 데이터프레임을 생성해보도록 하겠습니다.

In [44]:
test_mfccs = preprocess_dataset(test_x)
test_mfccs = np.array(test_mfccs)
test_mfccs = test_mfccs.reshape(-1, test_mfccs.shape[1], test_mfccs.shape[2], 1)

In [45]:
test_mfccs.shape

(200, 40, 12, 1)

이제 이 데이터프레임을 이용하여 라벨을 추론해보도록 하겠습니다.

In [46]:
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for wav in tqdm(iter(test_loader)):
            wav = wav.to(device)

            pred_logit = model(wav)
            pred_logit = pred_logit.argmax(dim=1, keepdim=True).squeeze(1)

            model_pred.extend(pred_logit.tolist())
    return model_pred

In [47]:
test_dataset = CustomDataset(X=test_mfccs, y= None, train_mode=False)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

In [48]:
# Validation Accuracy가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('data/saved/best_model2.pth')
model = CNNclassification().to(device)
model.load_state_dict(checkpoint)

# Inference
preds = predict(model, test_loader, device)
preds[0:5]

  0%|          | 0/20 [00:00<?, ?it/s]

[0, 5, 3, 6, 2]

In [49]:
len(preds)

200

In [51]:
test_wav['label'] = preds
test_wav = test_wav[['file_name', 'label']]

pred_df = test_wav.copy()
pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True]).reset_index(drop=True)
pred_df.head()

/tmp/ipykernel_11261/3358046839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_wav['label'] = preds


,file_name,label
0,003.wav,0
1,008.wav,9
2,010.wav,8
3,015.wav,8
4,024.wav,2


## 제출하기

submission에 예측한 값 preds를 넣어줍시다.

In [54]:
submission = pd.read_csv('data/sample_submission.csv')
submission['label'] = pred_df['label']
submission.head()

,file_name,label
0,003.wav,0
1,008.wav,9
2,010.wav,8
3,015.wav,8
4,024.wav,2


값이 배열안에 정상적으로 잘 들어간 것을 확인할 수 있습니다.

submission을 csv 파일로 저장합니다.   

index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다.   

정확한 채점을 위해 꼭 index=False를 넣어주세요.

In [55]:
submission.to_csv('data/saved/submit2.csv', index=False)

이렇게 생성된 submit2.csv 파일을 데이콘 대회 페이지에 업로드 & 제출하여 결과를 확인해보세요!

문제를 해결하기 위한 여러분의 방법을 코드 공유 게시판에 공유해주세요

좋아요와 댓글을 합산하여 가장 높은 점수를 얻으신 분께 데이콘 후드가 제공됩니다!